Things to track:
- Pitch
- Content of speech ("um", "uhh", laughing)
- Response time (hesitation)
- How specific is the answer

Consider normalizing speech



In [6]:
import pyaudio
import wave
import keyboard

# Initialize pyaudio
audio = pyaudio.PyAudio()

# Define stream parameters
CHANNELS = 1
FORMAT = pyaudio.paInt16
RATE = 44100
CHUNK = 1024

# Open stream
stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

print("Recording... Press 'q' to stop.")

frames = []

# Record audio until 'q' is pressed
while not keyboard.is_pressed('q'):
    data = stream.read(CHUNK)
    frames.append(data)

print("Finished recording.")

# Stop and close the stream
stream.stop_stream()
stream.close()
audio.terminate()

# Save the recorded data to a file
wf = wave.open("output.wav", 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(audio.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()


Recording... Press 'q' to stop.
Finished recording.


Testing Google's Cloud Speech-To-Text
https://cloud.google.com/speech-to-text/docs/transcribe-streaming-audio#speech-streaming-recognize-python

This transcribes streaming audio to text.

To install the library:
pip install --upgrade google-cloud-speech

Download the gcloud CLI: https://cloud.google.com/sdk/docs/install.

Configure the development environment. 
<img src="attachment:image.png" width="300"/>

For some reason you have to go here to accept terms of service with gcloud, before doing "gcloud init" command
https://console.cloud.google.com/terms/universal?pli=

You have to request access to use this, as it looks like 1 specific person will "own" the project. People can be added directly to the project.

Info:
Project name: truthinators
Project number: 399310020607
Project ID: truthinators

You have to enable the Speech to Text API for the specific project + add billing :(

API Request limits: 
https://cloud.google.com/speech-to-text/quotas
For our purposes, we should not be going over the API limitations (it's like... 480 hours of audio per day, 900 requests per 60 seconds).







In [18]:

# Imports the Google Cloud client library


from google.cloud import speech



def run_quickstart() -> speech.RecognizeResponse:
    # Instantiates a client
    client = speech.SpeechClient()

    # The name of the audio file to transcribe
    gcs_uri = "gs://cloud-samples-data/speech/brooklyn_bridge.raw"

    audio = speech.RecognitionAudio(uri=gcs_uri)

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
    )

    # Detects speech in the audio file
    response = client.recognize(config=config, audio=audio)

    for result in response.results:
        print(f"Transcript: {result.alternatives[0].transcript}")
run_quickstart()


Transcript: how old is the Brooklyn Bridge


In [37]:

import queue
import re
import sys

from google.cloud import speech

import pyaudio

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms


class MicrophoneStream:
    """Opens a recording stream as a generator yielding the audio chunks."""

    def __init__(self: object, rate: int = RATE, chunk: int = CHUNK) -> None:
        """The audio -- and generator -- is guaranteed to be on the main thread."""
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self: object) -> object:
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (mono) audio
            # https://goo.gl/z757pE
            channels=1,
            rate=self._rate,
            input=True,
            frames_per_buffer=self._chunk,
            # Run the audio stream asynchronously to fill the buffer object.
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(
        self: object,
        type: object,
        value: object,
        traceback: object,
    ) -> None:
        """Closes the stream, regardless of whether the connection was lost or not."""
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination.
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(
        self: object,
        in_data: object,
        frame_count: int,
        time_info: object,
        status_flags: object,
    ) -> object:
        """Continuously collect data from the audio stream, into the buffer.

        Args:
            in_data: The audio data as a bytes object
            frame_count: The number of frames captured
            time_info: The time information
            status_flags: The status flags

        Returns:
            The audio data as a bytes object
        """
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self: object) -> object:
        """Generates audio chunks from the stream of audio data in chunks.

        Args:
            self: The MicrophoneStream object

        Returns:
            A generator that outputs audio chunks.
        """
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of
            # data, and stop iteration if the chunk is None, indicating the
            # end of the audio stream.
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            # Now consume whatever other data's still buffered.
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b"".join(data)


def listen_print_save_loop(responses: object, name: str) -> str:
    """Iterates through server responses and prints them.

    The responses passed is a generator that will block until a response
    is provided by the server.

    Each response may contain multiple results, and each result may contain
    multiple alternatives; for details, see https://goo.gl/tjCPAU.  Here we
    print only the transcription for the top alternative of the top result.

    In this case, responses are provided for interim results as well. If the
    response is an interim one, print a line feed at the end of it, to allow
    the next result to overwrite it, until the response is a final one. For the
    final one, print a newline to preserve the finalized transcription.
    
    Transcript is also added to "transcription.txt"

    Args:
        responses: List of server responses

    Returns:
        The transcribed text.
    """
    num_chars_printed = 0
    final_transcript = ""

    with open(f'{name}.txt', "w") as file:
        for response in responses:
            if not response.results:
                continue

            # Process the first result in the response
            result = response.results[0]
            if not result.alternatives:
                continue

            # Extract the top alternative transcript
            transcript = result.alternatives[0].transcript

            # Handle interim results with overwrite
            overwrite_chars = " " * (num_chars_printed - len(transcript))

            if not result.is_final:
                sys.stdout.write(transcript + overwrite_chars + "\r")
                sys.stdout.flush()
                num_chars_printed = len(transcript)
            else:
                # Final result: print and save to file
                print(transcript + overwrite_chars)
                file.write(transcript + overwrite_chars + "\n")

                # Check for exit keywords
                if re.search(r"\b(exit|quit)\b", transcript, re.I):
                    print("Exiting...")
                    break

                # Reset character count and append to final transcript
                num_chars_printed = 0
                final_transcript += transcript + " "

    return final_transcript.strip()

def save_audio_to_file(responses: object, name: str) -> None:
    """Saves audio data to an output file.

    The responses passed is a generator that will block until a response
    is provided by the server.

    Args:
        responses: List of server responses
    """
    with open((f'{name}.raw'), "wb") as f:
        for response in responses:
            if not response.results:
                continue
            f.write(response.results[0].audio_content)


def main() -> None:
    """Transcribe speech from audio file."""
    # See http://g.co/cloud/speech/docs/languages
    # for a list of supported languages.
    language_code = "en-US"  # a BCP-47 language tag
    
    participant_name = input()

    client = speech.SpeechClient()
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code,
    )

    streaming_config = speech.StreamingRecognitionConfig(
        config=config, interim_results=True
    )

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (
            speech.StreamingRecognizeRequest(audio_content=content)
            for content in audio_generator
        )

        responses = client.streaming_recognize(streaming_config, requests)

        # Now, put the transcription responses to use.
        listen_print_save_loop(responses, participant_name)


if __name__ == "__main__":
    main()

hello
 this is a test file
 can I wait can I actively read this
 exit
Exiting...
